In [ ]:
import os
import sys
import random
import torch
import json
import librosa
import pdb
ROOT_DIR = os.path.dirname(os.path.abspath("../README.md"))
sys.path.append('{}/third_party/Matcha-TTS'.format(ROOT_DIR))
from cosyvoice.cli.cosyvoice import CosyVoice2
from cosyvoice.utils.file_utils import load_wav
from cosyvoice.utils.common import set_all_random_seed

max_val = 0.8

def generate_seed():
    seed = random.randint(1, 100000000)
    return seed

def postprocess(speech, top_db=60, hop_length=220, win_length=440):
    speech, _ = librosa.effects.trim(
        speech, top_db=top_db,
        frame_length=win_length,
        hop_length=hop_length
    )
    if speech.abs().max() > max_val:
        speech = speech / speech.abs().max() * max_val
    speech = torch.concat([speech, torch.zeros(1, int(24000 * 0.2))], dim=1)
    return speech

def main(tts_gen_kwargs):
    model = CosyVoice2("/home/tuwenming/Models/FunAudioLLM/CosyVoice2-0.5B")
    
    set_all_random_seed(generate_seed())
    result_tokens = []
    for item in model.inference_zero_shot_ttst(**tts_gen_kwargs):
        tokens = item["speech_token"]
        result_tokens += tokens
        print("len of tokens: ",len(tokens))
    print(f"len of result_tokens: {len(result_tokens)}")
    # pdb.set_trace()


if __name__ == '__main__':
    with open("/home/tuwenming/Datasets/VocalNet/VoiceAssistant-430K-vocalnet/VoiceAssistant-430K.json") as f:
        item = json.load(f)
    text = item[0]['conversations'][1]['value']
    prompt_speech_16k = postprocess(load_wav("../omni_speech/infer/common_voice_en_2586258.wav", 16000))
    prompt_text = "Romeo and Juliet might be the most famous act of William Shakespeare."
    tts_gen_kwargs = dict(
        tts_text=text,
        prompt_text=prompt_text,
        prompt_speech_16k=prompt_speech_16k,
        stream=False,
        speed=1.0,
    )
    main(tts_gen_kwargs)
